# MedNIST dataset using DenseNet Transfer Learning¶

## Introduction

This tutorial shows how to do 2d image classification example on MedNIST dataset using pretrained PyTorch model Densnet121 https://pytorch.org/vision/main/generated/torchvision.models.densenet121.html.

About MedNIST

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

MedNIST dataset is downloaded from the resources provided by the project MONAI: https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz

The dataset MedNIST has 58954 images of size (3, 64, 64) distributed into 6 classes (10000 images per class except for BreastMRI class which has 8954 images). Classes are AbdomenCT, BreastMRI, CXR, ChestCT, Hand, HeadCT. It has the structure:

└── MedNIST/

├── AbdomenCT/

└── BreastMRI/

└── CXR/

└── ChestCT/

└── Hand/

└── HeadCT/   


In this example, we load on the node a sampled dataset ( 500 or 1000 images) of MedNIST to illustrate the effectiveness of the transfer learning. The sampled dataset is made with a random selection of images and return a sampled datset with balanced classes, to avoid classification's bias.

## About the model

The model used is Densenet-121 model(“Densely Connected Convolutional Networks”) pretrained on ImageNet dataset. The Pytorch pretrained Densenet121 is used https://pytorch.org/vision/main/generated/torchvision.models.densenet121.html to perform image classification on the MedNIST dataset. 
The goal of this Densenet121 model is to predict the class of the image modality given the medical image.



## Setup the node

- From the folder fedbiomed, execute the command ./scripts/fedbiomed_run node add

- Select option 3 (mednist) to add MedNIST to the node
- Confirm mednist tags ['#MEDNIST', '#dataset'] by hitting "y" and ENTER
- Select the folder where MedNIST is downloaded (It will be downloaded if it is not found in the selected path)
Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
- Enter the amount's sample you want to run in your experiment.

- Check that your data has been added by executing ./scripts/fedbiomed_run node list
- Start the node using ./scripts/fedbiomed_run node start. Wait until you get Starting task manager.


## Start Fed-BioMed Researcher

We are now ready to start the researcher environment with the command source ./scripts/fedbiomed_run researcher start
, and open the Jupyter notebook.

To make sure that MedNIST dataset is loaded in the node we can send a request to the network to list the available dataset in the node. The list command should output an entry for mednist data.


In [ ]:
from fedbiomed.researcher.requests import Requests
req  = Requests()
req.list()

## Import of librairies 

In [ ]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms
from torchvision.models import densenet121
from torchvision import datasets, transforms, models
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage


## Classification using Transfer-learning 

### Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [ ]:
class MyTrainingPlan1(TorchTrainingPlan):

    def init_model(self, model_args):
       
        # Load the pre-trained DenseNet model
        model = models.densenet121(pretrained=False)
        
        # remove the classification layer 
        for param in model.features[:-1].parameters():
            param.requires_grad = False
        
        num_classes = model_args['num_classes'] # Change this to the number of classes in your dataset model_args section below
        num_ftrs = model.classifier.in_features
        model.classifier= nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
      
        return model

    def init_dependencies(self):
        return [
            "from torchvision import datasets, transforms, models",
            "import torch.optim as optim",
            "from torchvision.models import densenet121"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    # training data
    

    def training_data(self):
        
        # Custom torch Dataloader for MedNIST data
        print("dataset path",self.dataset_path)

        # Transform images and  do data augmentation 
        preprocess = transforms.Compose([
                transforms.Resize((224,224)),  
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(30),
                transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
    
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




In [ ]:
training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-4}, 
    'epochs': 2, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

model_args = {
    'num_classes': 6
}

In [ ]:


tags =  ['#MEDNIST', '#dataset']

rounds = 2 # adjsut the number of rounds 

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan1,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

# testing section 
from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1) 
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)

In [ ]:
#exp.training_plan().import_model('model')

In [ ]:
exp.run()

 
For example,  At the end of training experiment, I obtained

                      INFO - VALIDATION ON LOCAL UPDATES 
					 NODE_ID: NODE_41cd99c8-3571-4ab3-958e-6357ce31e91b 
					 Round 2 | Iteration: 1/1 (100%) | Samples: 100/100
 					 ACCURACY: 0.700000 
					 -

In [ ]:
#save model and  weights. You have the choice to import the model for other experiment 
exp.training_plan().export_model('./training_plan1_densenet_MedNIST')

In [ ]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "$tensorboard_dir"

## Upload more data and train the top layers 

You can load more data on a new node for the second experiment and train top layers of the denseNet model.
To cange the amount of data, you have to stop the previous node in the console by tapping CTL+C.

In this example, I run a second experiment with 1000 images.
Run an other node with 1000 images (as previously described above)


In [ ]:
from fedbiomed.researcher.requests import Requests
req  = Requests()
req.list()

## Partial fine-tuning: Use pretrained DenseNet and train Top layers with your data

In [ ]:
class MyTrainingPlan2(TorchTrainingPlan):

    def init_model(self, model_args):

        # Load the pre-trained DenseNet model
        model = models.densenet121(pretrained=True)
        
        # For example, let's freeze layers of the last dense block
        for param in model.features[:-3].parameters():
            param.requires_grad = False

        # Modify the classifier to match the number of classes in your dataset
        num_ftrs = model.classifier.in_features
        num_classes = model_args['num_classes'] 
        model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
           
            )
        
        return model

    def init_dependencies(self):
        return [
            "from torchvision import datasets, transforms, models",
            "import torch.optim as optim"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    def training_data(self):
        
        # Custom torch Dataloader for MedNIST data
        print("dataset path",self.dataset_path)
        preprocess = transforms.Compose([
                transforms.Resize((224,224)),  
                #transforms.RandomHorizontalFlip(p=0.5),
                #transforms.RandomVerticalFlip(p=0.5),
                #transforms.RandomRotation(30),
                #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)



    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




In [ ]:
training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-4}, 
    'epochs': 1, # you can increase the epoch's number =10
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}
model_args={
    'num_classes': 6
}
tags =  ['#MEDNIST', '#dataset']
rounds = 1  # you can increase the rounds's number 

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan2,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1)
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)
    

In [ ]:
exp.run()

In [ ]:
For example,  At the end of training experiment, I obtained

                    fedbiomed INFO - VALIDATION ON LOCAL UPDATES 
					 NODE_ID: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Round 1 | Iteration: 1/1 (100%) | Samples: 100/100
 					 ACCURACY: 0.980000 
					 ---------

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "$tensorboard_dir"

## Save and export your model 

You can save the TrainingPlan experiment and the fine-tune model by executing the command below :

In [ ]:
exp.training_plan().export_model('./training_plan2_densenet_MedNIST')

In [ ]:
# save your model ( all layers model of te training experiment)
remote_model = exp.training_plan().model()
torch.save(remote_model, './training_plan2_model')

In [ ]:
#from torchvision import models

#torch.save(models.densenet121(pretrained=True).state_dict(), './model_training_plan_2')

## Import your model and parameters 

In [ ]:
your_model = torch.load('./training_plan2_model')

In [ ]:
# load your parameters (tensors's values of your tuned-model)
tuned_model= torch.load('./training_plan2_densenet_MedNIST')

In [ ]:
# In a new TrainingPlan experiment you could import your tuned-model 
exp.training_plan().import_model('./training_plan2_densenet_MedNIST')

### This part needs confirmation, tests,( and agreements to load parameters ? ) 

In [ ]:
#remote_model = remote_experiment.training_plan().model()
tuned_model.load_state_dict(remote_experiment.aggregated_params()[rounds - 1]['params'])

In [ ]:
tuned_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])
#tuned_model.load_state_dict(remote_experiment.aggregated_params()[rounds - 1]['params'])

In [ ]:
tuned_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])